# Knowledge Graph Question Answering System for GW Courses

This notebook implements a **comprehensive Knowledge Graph QA system** with:
- Knowledge Graph Construction (Courses, Professors, Topics, Prerequisites)
- Multi-hop Reasoning Training Data
- Graph Attention Network (GAT) for Graph Retrieval
- Hybrid Retrieval + Generation Architecture (RAG)
- Comprehensive Evaluation Metrics
- Prerequisites and Topic Extraction
- Structured Output Format
- Data Augmentation
- Performance Monitoring

**Project Goal**: Build an intelligent question-answering system over a custom knowledge graph of GW courses with multi-hop reasoning capabilities.


## 1. Setup Environment


In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install evaluate
!pip install torch-geometric networkx pandas numpy scikit-learn
!pip install nltk spacy
!python -m spacy download en_core_web_sm


In [ ]:
import pandas as pd
import numpy as np
import json
import re
import networkx as nx
from collections import defaultdict, Counter
from typing import Dict, List, Tuple, Set, Optional
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from datasets import load_dataset, Dataset
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    pass

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except:
    print("Warning: spaCy model not loaded. Topic extraction may be limited.")
    nlp = None

print("✅ Environment setup complete")


## 2. Load and Prepare Data


In [ ]:
# Load course data
courses_df = pd.read_csv("data/spring_2026_courses.csv")
bulletin_df = pd.read_csv("data/bulletin_courses.csv")

print(f"Loaded {len(courses_df)} course sections")
print(f"Loaded {len(bulletin_df)} course descriptions")

# Create course code to description mapping
course_descriptions = {}
for _, row in bulletin_df.iterrows():
    code = str(row['course_code']).strip()
    desc = str(row.get('description', '')).strip()
    if desc and desc != 'nan':
        course_descriptions[code] = desc

print(f"✅ Data loaded: {len(course_descriptions)} course descriptions available")


## 3. Prerequisites and Topic Extraction


In [ ]:
def extract_prerequisites(description: str) -> List[str]:
    """Extract prerequisite course codes from description."""
    if not description or description == 'nan':
        return []
    
    prerequisites = []
    # Pattern: "Prerequisites: CSCI 1112" or "Prerequisite: CSCI 1112"
    patterns = [
        r'[Pp]rerequisite[s]?[:\s]+([A-Z]{2,}\s+\d{4})',
        r'[Pp]rerequisite[s]?[:\s]+([A-Z]{2,}\s+\d{4}[A-Z]?)',
        r'([A-Z]{2,}\s+\d{4}[A-Z]?)\s+with\s+a\s+minimum\s+grade',
        r'([A-Z]{2,}\s+\d{4}[A-Z]?)\s+or\s+([A-Z]{2,}\s+\d{4}[A-Z]?)',
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, description)
        if matches:
            if isinstance(matches[0], tuple):
                prerequisites.extend([m for m in matches if m])
            else:
                prerequisites.extend(matches)
    
    # Clean and normalize
    prerequisites = [p.strip() for p in prerequisites if p.strip()]
    return list(set(prerequisites))

def extract_topics(description: str, course_code: str) -> List[str]:
    """Extract topics from course description using NLP."""
    if not description or description == 'nan':
        return []
    
    topics = []
    
    # Common CS topics/keywords
    cs_topics = [
        'machine learning', 'deep learning', 'neural networks', 'computer vision',
        'natural language processing', 'nlp', 'data structures', 'algorithms',
        'database', 'software engineering', 'operating systems', 'networks',
        'distributed systems', 'security', 'cryptography', 'web development',
        'mobile development', 'artificial intelligence', 'ai', 'robotics',
        'graphics', 'game development', 'cloud computing', 'parallel computing',
        'compilers', 'programming languages', 'theory', 'optimization'
    ]
    
    description_lower = description.lower()
    for topic in cs_topics:
        if topic in description_lower:
            topics.append(topic)
    
    # Use spaCy for named entity recognition if available
    if nlp:
        doc = nlp(description)
        # Extract technical terms (nouns and noun phrases)
        for chunk in doc.noun_chunks:
            text = chunk.text.lower()
            if len(text) > 3 and text not in stopwords.words('english'):
                # Filter for technical terms
                if any(keyword in text for keyword in ['algorithm', 'system', 'structure', 'model', 'framework']):
                    topics.append(text)
    
    return list(set(topics))

# Extract prerequisites and topics for all courses
course_prerequisites = {}
course_topics = {}

for course_code, description in course_descriptions.items():
    course_prerequisites[course_code] = extract_prerequisites(description)
    course_topics[course_code] = extract_topics(description, course_code)

# Print statistics
total_with_prereqs = sum(1 for v in course_prerequisites.values() if v)
total_with_topics = sum(1 for v in course_topics.values() if v)

print(f"✅ Extracted prerequisites for {total_with_prereqs} courses")
print(f"✅ Extracted topics for {total_with_topics} courses")
print(f"\nSample prerequisites: {dict(list(course_prerequisites.items())[:5])}")
print(f"\nSample topics: {dict(list(course_topics.items())[:5])}")


In [ ]:
class KnowledgeGraph:
    """Knowledge Graph for GW Courses with nodes and edges."""
    
    def __init__(self):
        self.graph = nx.DiGraph()  # Directed graph
        self.course_nodes = {}  # course_code -> node_id
        self.professor_nodes = {}  # professor_name -> node_id
        self.topic_nodes = {}  # topic -> node_id
        self.node_features = {}  # node_id -> features
        self.edge_types = {}  # (source, target) -> edge_type
        self.node_id_counter = 0
        
    def add_node(self, node_type: str, node_id: str, features: Dict = None):
        """Add a node to the graph."""
        if node_id not in self.graph:
            self.graph.add_node(node_id, node_type=node_type, **{**(features or {})})
            self.node_features[node_id] = features or {}
            return True
        return False
    
    def add_edge(self, source: str, target: str, edge_type: str, weight: float = 1.0):
        """Add an edge to the graph."""
        if source in self.graph and target in self.graph:
            self.graph.add_edge(source, target, edge_type=edge_type, weight=weight)
            self.edge_types[(source, target)] = edge_type
            return True
        return False
    
    def build_from_data(self, courses_df: pd.DataFrame, course_descriptions: Dict,
                       course_prerequisites: Dict, course_topics: Dict):
        """Build knowledge graph from course data."""
        print("Building knowledge graph...")
        
        # Add course nodes
        unique_courses = courses_df['subject_code'].unique()
        for course_code in unique_courses:
            course_code = str(course_code).strip()
            if course_code and course_code != 'nan':
                node_id = f"course_{course_code}"
                description = course_descriptions.get(course_code, "")
                features = {
                    'code': course_code,
                    'description': description,
                    'has_prerequisites': len(course_prerequisites.get(course_code, [])) > 0,
                    'topics': course_topics.get(course_code, [])
                }
                self.add_node('course', node_id, features)
                self.course_nodes[course_code] = node_id
        
        # Add professor nodes
        unique_professors = courses_df['instructor'].dropna().unique()
        for prof in unique_professors:
            prof = str(prof).strip()
            if prof and prof != 'nan':
                node_id = f"prof_{prof.replace(' ', '_').replace(',', '')}"
                if self.add_node('professor', node_id, {'name': prof}):
                    self.professor_nodes[prof] = node_id
        
        # Add topic nodes
        all_topics = set()
        for topics in course_topics.values():
            all_topics.update(topics)
        
        for topic in all_topics:
            node_id = f"topic_{topic.replace(' ', '_')}"
            if self.add_node('topic', node_id, {'name': topic}):
                self.topic_nodes[topic] = node_id
        
        # Add edges: taught_by
        for _, row in courses_df.iterrows():
            course_code = str(row['subject_code']).strip()
            prof = str(row.get('instructor', '')).strip()
            
            if course_code in self.course_nodes and prof in self.professor_nodes:
                course_node = self.course_nodes[course_code]
                prof_node = self.professor_nodes[prof]
                self.add_edge(course_node, prof_node, 'taught_by')
        
        # Add edges: prerequisite
        for course_code, prereqs in course_prerequisites.items():
            if course_code in self.course_nodes:
                course_node = self.course_nodes[course_code]
                for prereq_code in prereqs:
                    prereq_node_id = f"course_{prereq_code}"
                    if prereq_node_id in self.graph:
                        self.add_edge(course_node, prereq_node_id, 'prerequisite')
        
        # Add edges: covers_topic
        for course_code, topics in course_topics.items():
            if course_code in self.course_nodes:
                course_node = self.course_nodes[course_code]
                for topic in topics:
                    topic_node_id = f"topic_{topic.replace(' ', '_')}"
                    if topic_node_id in self.graph:
                        self.add_edge(course_node, topic_node_id, 'covers_topic')
        
        print(f"✅ Graph built: {self.graph.number_of_nodes()} nodes, {self.graph.number_of_edges()} edges")
        print(f"   - Courses: {len(self.course_nodes)}")
        print(f"   - Professors: {len(self.professor_nodes)}")
        print(f"   - Topics: {len(self.topic_nodes)}")
    
    def get_subgraph(self, start_nodes: List[str], max_hops: int = 2) -> nx.DiGraph:
        """Get subgraph starting from given nodes with max_hops depth."""
        subgraph_nodes = set(start_nodes)
        
        for _ in range(max_hops):
            new_nodes = set()
            for node in subgraph_nodes:
                # Get neighbors (both incoming and outgoing)
                new_nodes.update(self.graph.successors(node))
                new_nodes.update(self.graph.predecessors(node))
            subgraph_nodes.update(new_nodes)
        
        return self.graph.subgraph(subgraph_nodes)
    
    def find_paths(self, source: str, target: str, max_length: int = 3) -> List[List[str]]:
        """Find all paths from source to target."""
        try:
            paths = list(nx.all_simple_paths(self.graph, source, target, cutoff=max_length))
            return paths
        except:
            return []

# Build knowledge graph
kg = KnowledgeGraph()
kg.build_from_data(courses_df, course_descriptions, course_prerequisites, course_topics)


In [ ]:
class GraphAttentionNetwork(nn.Module):
    """Graph Attention Network for scoring relevant graph paths."""
    
    def __init__(self, input_dim: int, hidden_dim: int = 128, num_heads: int = 4, num_layers: int = 2):
        super().__init__()
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        
        # First layer
        self.gat_layers.append(GATConv(input_dim, hidden_dim, heads=num_heads, dropout=0.1, concat=True))
        
        # Intermediate layers
        for _ in range(num_layers - 2):
            self.gat_layers.append(GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads, dropout=0.1, concat=True))
        
        # Final layer (no concatenation)
        if num_layers > 1:
            self.gat_layers.append(GATConv(hidden_dim * num_heads, hidden_dim, heads=1, dropout=0.1, concat=False))
        
        self.output_proj = nn.Linear(hidden_dim, 1)  # Score output
        
    def forward(self, x, edge_index, edge_attr=None):
        """Forward pass through GAT layers."""
        for i, gat_layer in enumerate(self.gat_layers):
            x = gat_layer(x, edge_index)
            if i < len(self.gat_layers) - 1:
                x = torch.relu(x)
        
        # Output score
        scores = self.output_proj(x).squeeze(-1)
        return scores, x

class GraphRetriever:
    """Retriever that uses GAT to find relevant graph subgraphs for queries."""
    
    def __init__(self, knowledge_graph: KnowledgeGraph, gat_model: Optional[GraphAttentionNetwork] = None):
        self.kg = knowledge_graph
        self.gat_model = gat_model
        
    def retrieve_subgraph(self, query: str, query_entities: List[str], max_hops: int = 2) -> nx.DiGraph:
        """Retrieve relevant subgraph for a query."""
        # Find starting nodes from query entities
        start_nodes = []
        for entity in query_entities:
            # Try to match course codes
            if entity in self.kg.course_nodes:
                start_nodes.append(self.kg.course_nodes[entity])
            # Try to match professors
            for prof_name, node_id in self.kg.professor_nodes.items():
                if entity.lower() in prof_name.lower() or prof_name.lower() in entity.lower():
                    start_nodes.append(node_id)
            # Try to match topics
            for topic, node_id in self.kg.topic_nodes.items():
                if entity.lower() in topic.lower():
                    start_nodes.append(node_id)
        
        if not start_nodes:
            # If no entities found, return empty subgraph
            return nx.DiGraph()
        
        # Get subgraph
        subgraph = self.kg.get_subgraph(start_nodes, max_hops=max_hops)
        return subgraph
    
    def format_subgraph_context(self, subgraph: nx.DiGraph) -> str:
        """Format subgraph as text context for LLM."""
        if subgraph.number_of_nodes() == 0:
            return "No relevant graph information found."
        
        context_parts = []
        
        # Group by edge type
        edges_by_type = defaultdict(list)
        for u, v, data in subgraph.edges(data=True):
            edge_type = data.get('edge_type', 'unknown')
            edges_by_type[edge_type].append((u, v))
        
        # Format prerequisite relationships
        if 'prerequisite' in edges_by_type:
            prereqs = []
            for u, v in edges_by_type['prerequisite']:
                course_u = subgraph.nodes[u].get('code', u)
                course_v = subgraph.nodes[v].get('code', v)
                prereqs.append(f"{course_v} is a prerequisite for {course_u}")
            if prereqs:
                context_parts.append("Prerequisites: " + "; ".join(prereqs[:10]))
        
        # Format taught_by relationships
        if 'taught_by' in edges_by_type:
            taught_by = []
            for u, v in edges_by_type['taught_by']:
                course = subgraph.nodes[u].get('code', u)
                prof = subgraph.nodes[v].get('name', v)
                taught_by.append(f"{course} is taught by {prof}")
            if taught_by:
                context_parts.append("Instructors: " + "; ".join(taught_by[:10]))
        
        # Format covers_topic relationships
        if 'covers_topic' in edges_by_type:
            topics = []
            for u, v in edges_by_type['covers_topic']:
                course = subgraph.nodes[u].get('code', u)
                topic = subgraph.nodes[v].get('name', v)
                topics.append(f"{course} covers {topic}")
            if topics:
                context_parts.append("Topics: " + "; ".join(topics[:10]))
        
        return "\n".join(context_parts) if context_parts else "Graph context available."

# Initialize retriever
retriever = GraphRetriever(kg)
print("✅ Graph Retriever initialized")


In [ ]:
def extract_entities_from_query(query: str) -> List[str]:
    """Extract potential entities (course codes, professor names) from query."""
    entities = []
    
    # Extract course codes (e.g., "CSCI 6212")
    course_pattern = r'([A-Z]{2,}\s+\d{4}[A-Z]?)'
    course_matches = re.findall(course_pattern, query)
    entities.extend(course_matches)
    
    # Extract topic mentions
    for topic in kg.topic_nodes.keys():
        if topic.lower() in query.lower():
            entities.append(topic)
    
    return list(set(entities))

def generate_multi_hop_questions(kg: KnowledgeGraph, num_examples: int = 100) -> List[Dict]:
    """Generate multi-hop reasoning questions from the knowledge graph."""
    questions = []
    
    # Type 1: Prerequisite chain questions
    # "Which courses should I take to prepare for X if I've completed Y?"
    courses_with_prereqs = [code for code, prereqs in course_prerequisites.items() if prereqs]
    
    for _ in range(min(num_examples // 4, len(courses_with_prereqs))):
        target_course = np.random.choice(courses_with_prereqs)
        prereqs = course_prerequisites[target_course]
        if prereqs:
            completed_course = np.random.choice(prereqs)
            
            # Find all prerequisites in the chain
            all_prereqs = []
            def get_all_prereqs(course):
                if course in course_prerequisites:
                    for p in course_prerequisites[course]:
                        if p not in all_prereqs:
                            all_prereqs.append(p)
                            get_all_prereqs(p)
            
            get_all_prereqs(target_course)
            
            query = f"Which courses should I take to prepare for {target_course} if I've completed {completed_course}?"
            
            # Build answer
            remaining_prereqs = [p for p in all_prereqs if p != completed_course]
            if remaining_prereqs:
                answer = f"To prepare for {target_course}, you should also take: {', '.join(remaining_prereqs[:5])}."
            else:
                answer = f"After completing {completed_course}, you are ready to take {target_course}."
            
            # Get graph context
            entities = extract_entities_from_query(query)
            subgraph = retriever.retrieve_subgraph(query, entities, max_hops=3)
            graph_context = retriever.format_subgraph_context(subgraph)
            
            questions.append({
                'query': query,
                'answer': answer,
                'graph_context': graph_context,
                'reasoning_path': f"{completed_course} -> {target_course}",
                'type': 'prerequisite_chain'
            })
    
    # Type 2: Professor intersection questions
    # "Which professors teach courses that are prerequisites for both X and Y?"
    courses_list = list(kg.course_nodes.keys())[:50]  # Limit for performance
    
    for _ in range(min(num_examples // 4, 50)):
        if len(courses_list) < 2:
            break
        course1, course2 = np.random.choice(courses_list, 2, replace=False)
        
        # Find professors teaching prerequisites of both
        prereqs1 = set(course_prerequisites.get(course1, []))
        prereqs2 = set(course_prerequisites.get(course2, []))
        common_prereqs = prereqs1.intersection(prereqs2)
        
        if common_prereqs:
            query = f"Which professors teach courses that are prerequisites for both {course1} and {course2}?"
            
            # Find professors teaching common prerequisites
            profs = []
            for _, row in courses_df.iterrows():
                if str(row['subject_code']).strip() in common_prereqs:
                    prof = str(row.get('instructor', '')).strip()
                    if prof and prof != 'nan':
                        profs.append(prof)
            
            if profs:
                answer = f"Professors teaching prerequisites for both courses include: {', '.join(set(profs)[:5])}."
            else:
                answer = f"No professors found teaching common prerequisites for {course1} and {course2}."
            
            entities = extract_entities_from_query(query)
            subgraph = retriever.retrieve_subgraph(query, entities, max_hops=3)
            graph_context = retriever.format_subgraph_context(subgraph)
            
            questions.append({
                'query': query,
                'answer': answer,
                'graph_context': graph_context,
                'reasoning_path': f"{course1} ∩ {course2} prerequisites",
                'type': 'professor_intersection'
            })
    
    # Type 3: Topic-based questions
    # "What courses cover machine learning and are prerequisites for computer vision?"
    topics_list = list(kg.topic_nodes.keys())[:20]
    
    for _ in range(min(num_examples // 4, 30)):
        if len(topics_list) < 2:
            break
        topic1, topic2 = np.random.choice(topics_list, 2, replace=False)
        
        # Find courses covering topic1
        courses_topic1 = [code for code, topics in course_topics.items() if topic1 in topics]
        # Find courses that are prerequisites for courses covering topic2
        courses_topic2 = [code for code, topics in course_topics.items() if topic2 in topics]
        
        relevant_courses = []
        for c1 in courses_topic1:
            prereqs = course_prerequisites.get(c1, [])
            if any(c2 in prereqs for c2 in courses_topic2):
                relevant_courses.append(c1)
        
        if relevant_courses:
            query = f"What courses cover {topic1} and are prerequisites for courses covering {topic2}?"
            answer = f"Courses covering {topic1} that are prerequisites for {topic2} courses include: {', '.join(relevant_courses[:5])}."
            
            entities = extract_entities_from_query(query)
            subgraph = retriever.retrieve_subgraph(query, entities, max_hops=3)
            graph_context = retriever.format_subgraph_context(subgraph)
            
            questions.append({
                'query': query,
                'answer': answer,
                'graph_context': graph_context,
                'reasoning_path': f"{topic1} -> {topic2}",
                'type': 'topic_based'
            })
    
    # Type 4: Multi-hop path questions
    # "What is the path from course X to course Y through prerequisites?"
    for _ in range(min(num_examples // 4, 30)):
        if len(courses_list) < 2:
            break
        source, target = np.random.choice(courses_list, 2, replace=False)
        
        source_node = kg.course_nodes.get(source)
        target_node = kg.course_nodes.get(target)
        
        if source_node and target_node:
            paths = kg.find_paths(source_node, target_node, max_length=4)
            if paths:
                path = paths[0]  # Take first path
                path_courses = [kg.graph.nodes[n].get('code', n) for n in path if kg.graph.nodes[n].get('node_type') == 'course']
                
                query = f"What is the prerequisite path from {source} to {target}?"
                answer = f"The path from {source} to {target} is: {' -> '.join(path_courses)}."
                
                entities = extract_entities_from_query(query)
                subgraph = retriever.retrieve_subgraph(query, entities, max_hops=4)
                graph_context = retriever.format_subgraph_context(subgraph)
                
                questions.append({
                    'query': query,
                    'answer': answer,
                    'graph_context': graph_context,
                    'reasoning_path': ' -> '.join(path_courses),
                    'type': 'multi_hop_path'
                })
    
    return questions

# Generate multi-hop questions
print("Generating multi-hop reasoning questions...")
multi_hop_questions = generate_multi_hop_questions(kg, num_examples=200)
print(f"✅ Generated {len(multi_hop_questions)} multi-hop questions")
print(f"\nSample questions:")
for i, q in enumerate(multi_hop_questions[:3]):
    print(f"\n{i+1}. {q['query']}")
    print(f"   Answer: {q['answer']}")
    print(f"   Type: {q['type']}")


In [ ]:
def create_rag_training_example(query: str, answer: str, graph_context: str, 
                                reasoning_path: str = "", include_structure: bool = True) -> Dict:
    """Create a RAG training example with graph context."""
    
    # Build system message with graph context
    system_content = """You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026.
You have access to a knowledge graph with course relationships, prerequisites, instructors, and topics.
Use the provided graph context to answer questions accurately."""
    
    # Build user message with graph context
    if graph_context and graph_context != "No relevant graph information found.":
        user_content = f"""Graph Context:
{graph_context}

Question: {query}"""
    else:
        user_content = f"Question: {query}"
    
    # Build assistant response with structured format
    if include_structure and reasoning_path:
        assistant_content = f"""Reasoning Path: {reasoning_path}

Answer: {answer}"""
    else:
        assistant_content = answer
    
    return {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ],
        "graph_context": graph_context,
        "reasoning_path": reasoning_path,
        "query_type": "multi_hop" if reasoning_path else "simple"
    }

# Load existing simple Q&A data
existing_dataset = []
try:
    with open("data/course_finetune.jsonl", 'r') as f:
        for line in f:
            if line.strip():
                existing_dataset.append(json.loads(line))
    print(f"Loaded {len(existing_dataset)} existing examples")
except:
    print("No existing dataset found, starting fresh")

# Convert existing examples to RAG format (without graph context for simple ones)
rag_dataset = []
for example in existing_dataset[:500]:  # Limit to avoid too much data
    messages = example.get('messages', [])
    if len(messages) >= 3:
        user_msg = messages[1].get('content', '')
        assistant_msg = messages[2].get('content', '')
        
        # Extract entities and get graph context
        entities = extract_entities_from_query(user_msg)
        if entities:
            subgraph = retriever.retrieve_subgraph(user_msg, entities, max_hops=2)
            graph_context = retriever.format_subgraph_context(subgraph)
        else:
            graph_context = ""
        
        rag_example = create_rag_training_example(
            user_msg, assistant_msg, graph_context, 
            reasoning_path="", include_structure=False
        )
        rag_dataset.append(rag_example)

# Add multi-hop questions
for q in multi_hop_questions:
    rag_example = create_rag_training_example(
        q['query'], q['answer'], q['graph_context'],
        reasoning_path=q.get('reasoning_path', ''), include_structure=True
    )
    rag_dataset.append(rag_example)

print(f"✅ Created RAG dataset with {len(rag_dataset)} examples")
print(f"   - Simple Q&A: {len([x for x in rag_dataset if x['query_type'] == 'simple'])}")
print(f"   - Multi-hop: {len([x for x in rag_dataset if x['query_type'] == 'multi_hop'])}")

# Save dataset
output_file = "data/course_finetune_kg_rag.jsonl"
with open(output_file, 'w') as f:
    for example in rag_dataset:
        f.write(json.dumps(example) + "\n")
print(f"✅ Saved dataset to {output_file}")


In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Model configuration
max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True

print("Loading model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Setup chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

print("✅ Model loaded")


In [ ]:
# Configure LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA configured")


## 9. Prepare Dataset for Training


In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files="data/course_finetune_kg_rag.jsonl", split="train")

def formatting_prompts_func(examples):
    """Format dataset with chat template."""
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) 
             for convo in convos]
    return {"text": texts}

# Format dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Train/validation split (80/20)
dataset = dataset.train_test_split(test_size=0.2, seed=3407)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"✅ Dataset prepared:")
print(f"   Train examples: {len(train_dataset)}")
print(f"   Validation examples: {len(eval_dataset)}")
print(f"\nSample training example:")
print(train_dataset[0]["text"][:500] + "...")


## 10. Training Configuration with Evaluation Metrics


In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import EarlyStoppingCallback
import evaluate

# Load evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Compute evaluation metrics."""
    predictions, labels = eval_pred
    
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute BLEU
    bleu_results = bleu_metric.compute(
        predictions=decoded_preds,
        references=[[ref] for ref in decoded_labels]
    )
    
    # Compute ROUGE
    rouge_results = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    
    return {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
    }

# Training configuration
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=SFTConfig(
        # Batch size
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        
        # Learning rate
        learning_rate=1e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        
        # Training duration
        num_train_epochs=5,
        max_steps=-1,
        
        # Optimization
        optim="adamw_8bit",
        weight_decay=0.01,
        adam_beta1=0.9,
        adam_beta2=0.999,
        
        # Evaluation and logging
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=200,
        logging_steps=10,
        report_to="none",
        
        # Output
        output_dir="outputs_kg_qa",
        seed=3407,
        fp16=True,
        bf16=False,
        
        # Early stopping
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        
        # Metrics
        # Note: compute_metrics will be called during evaluation
    ),
)

# Add early stopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001,
)
trainer.add_callback(early_stopping)

print("✅ Training configuration complete")


## 11. Train Model


In [ ]:
# Check GPU memory
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Train
print("\nStarting training...")
trainer_stats = trainer.train()

# Training statistics
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print(f"\n✅ Training completed!")
print(f"Runtime: {trainer_stats.metrics['train_runtime']:.2f} seconds ({trainer_stats.metrics['train_runtime']/60:.2f} minutes)")
print(f"Peak reserved memory: {used_memory} GB ({used_percentage}%)")
print(f"Training memory: {used_memory_for_lora} GB")
print(f"Final training loss: {trainer_stats.metrics.get('train_loss', 'N/A')}")


## 12. Evaluation Framework


In [ ]:
def exact_match(prediction: str, reference: str) -> bool:
    """Check if prediction exactly matches reference."""
    return prediction.strip().lower() == reference.strip().lower()

def f1_score(prediction: str, reference: str) -> float:
    """Compute F1 score between prediction and reference."""
    pred_tokens = set(prediction.lower().split())
    ref_tokens = set(reference.lower().split())
    
    if len(pred_tokens) == 0 or len(ref_tokens) == 0:
        return 0.0
    
    intersection = pred_tokens.intersection(ref_tokens)
    if len(intersection) == 0:
        return 0.0
    
    precision = len(intersection) / len(pred_tokens)
    recall = len(intersection) / len(ref_tokens)
    
    if precision + recall == 0:
        return 0.0
    
    return 2 * (precision * recall) / (precision + recall)

def evaluate_qa_predictions(predictions: List[str], references: List[str]) -> Dict:
    """Evaluate QA predictions with multiple metrics."""
    em_scores = [exact_match(p, r) for p, r in zip(predictions, references)]
    f1_scores = [f1_score(p, r) for p, r in zip(predictions, references)]
    
    # Compute BLEU and ROUGE
    bleu_results = bleu_metric.compute(
        predictions=predictions,
        references=[[ref] for ref in references]
    )
    rouge_results = rouge_metric.compute(
        predictions=predictions,
        references=references
    )
    
    return {
        "exact_match": np.mean(em_scores),
        "f1": np.mean(f1_scores),
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
    }

# Create test set with complex queries
test_queries = [
    {
        "query": "Which courses should I take to prepare for CSCI 6364 if I've completed CSCI 1112?",
        "expected_entities": ["CSCI 6364", "CSCI 1112"],
        "type": "prerequisite_chain"
    },
    {
        "query": "Who teaches Machine Learning?",
        "expected_entities": ["machine learning"],
        "type": "simple"
    },
    {
        "query": "What courses cover computer vision and are prerequisites for deep learning courses?",
        "expected_entities": ["computer vision", "deep learning"],
        "type": "topic_based"
    },
    {
        "query": "Tell me about CSCI 1012.",
        "expected_entities": ["CSCI 1012"],
        "type": "simple"
    },
    {
        "query": "Which professors teach courses that are prerequisites for both CSCI 6364 and CSCI 6444?",
        "expected_entities": ["CSCI 6364", "CSCI 6444"],
        "type": "professor_intersection"
    }
]

print(f"✅ Evaluation framework ready with {len(test_queries)} test queries")


## 13. Inference Pipeline with Graph Retrieval


In [ ]:
FastLanguageModel.for_inference(model)

def answer_with_graph_retrieval(query: str, max_new_tokens: int = 256) -> Dict:
    """Answer query using graph retrieval + LLM generation."""
    
    # Step 1: Extract entities from query
    entities = extract_entities_from_query(query)
    
    # Step 2: Retrieve relevant subgraph
    subgraph = retriever.retrieve_subgraph(query, entities, max_hops=3)
    graph_context = retriever.format_subgraph_context(subgraph)
    
    # Step 3: Build prompt with graph context
    system_content = """You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026.
You have access to a knowledge graph with course relationships, prerequisites, instructors, and topics.
Use the provided graph context to answer questions accurately."""
    
    if graph_context and graph_context != "No relevant graph information found.":
        user_content = f"""Graph Context:
{graph_context}

Question: {query}"""
    else:
        user_content = f"Question: {query}"
    
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ]
    
    # Step 4: Generate answer
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    attention_mask = torch.ones_like(inputs)
    
    outputs = model.generate(
        input_ids=inputs,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,
        top_p=0.9,
    )
    
    # Decode answer
    output_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    
    return {
        "query": query,
        "answer": output_text.strip(),
        "graph_context": graph_context,
        "entities_found": entities,
        "subgraph_size": subgraph.number_of_nodes() if subgraph else 0
    }

# Test inference
print("Testing inference pipeline...\n")
for i, test_query in enumerate(test_queries[:3], 1):
    print(f"{'='*60}")
    print(f"Test {i}: {test_query['query']}")
    print(f"{'='*60}")
    
    result = answer_with_graph_retrieval(test_query['query'])
    
    print(f"Entities found: {result['entities_found']}")
    print(f"Subgraph nodes: {result['subgraph_size']}")
    print(f"\nGraph Context:\n{result['graph_context'][:200]}...")
    print(f"\nAnswer:\n{result['answer']}")
    print()


## 14. Performance Monitoring and Metrics


In [ ]:
# Evaluate on test set
print("Evaluating on test queries...\n")

predictions = []
references = []
graph_retrieval_stats = {
    "total_queries": 0,
    "queries_with_graph_context": 0,
    "avg_subgraph_size": [],
    "entities_extracted": 0
}

for test_query in test_queries:
    result = answer_with_graph_retrieval(test_query['query'])
    predictions.append(result['answer'])
    
    # For evaluation, we'd need ground truth answers
    # For now, we'll use a placeholder
    references.append("")  # Would be actual ground truth
    
    # Track graph retrieval stats
    graph_retrieval_stats["total_queries"] += 1
    if result['graph_context'] and result['graph_context'] != "No relevant graph information found.":
        graph_retrieval_stats["queries_with_graph_context"] += 1
    graph_retrieval_stats["avg_subgraph_size"].append(result['subgraph_size'])
    graph_retrieval_stats["entities_extracted"] += len(result['entities_found'])

# Print statistics
print("Graph Retrieval Statistics:")
print(f"  Total queries: {graph_retrieval_stats['total_queries']}")
print(f"  Queries with graph context: {graph_retrieval_stats['queries_with_graph_context']}")
print(f"  Average subgraph size: {np.mean(graph_retrieval_stats['avg_subgraph_size']):.2f}")
print(f"  Total entities extracted: {graph_retrieval_stats['entities_extracted']}")
print(f"  Average entities per query: {graph_retrieval_stats['entities_extracted'] / graph_retrieval_stats['total_queries']:.2f}")

# Note: Full evaluation with ground truth would require labeled test set
print("\n✅ Performance monitoring complete")


## 15. Data Augmentation for Synthetic Questions


In [ ]:
def augment_question(query: str, answer: str) -> List[Dict]:
    """Generate variations of a question for data augmentation."""
    variations = []
    
    # Variation 1: Paraphrase
    # Simple paraphrasing (in production, use a paraphrase model)
    if "which courses" in query.lower():
        variations.append({
            "query": query.replace("Which courses", "What courses"),
            "answer": answer,
            "type": "paraphrase"
        })
    
    # Variation 2: Question type change
    if "who teaches" in query.lower():
        variations.append({
            "query": query.replace("Who teaches", "Which professor teaches"),
            "answer": answer,
            "type": "question_type"
        })
    
    # Variation 3: Add context
    if "if I've completed" in query.lower():
        variations.append({
            "query": query.replace("if I've completed", "assuming I have completed"),
            "answer": answer,
            "type": "context_variation"
        })
    
    return variations

def generate_synthetic_questions_from_graph(kg: KnowledgeGraph, num_synthetic: int = 50) -> List[Dict]:
    """Generate synthetic questions by exploring the graph structure."""
    synthetic = []
    
    # Generate questions by following graph paths
    courses_list = list(kg.course_nodes.keys())[:30]
    
    for _ in range(num_synthetic):
        # Random walk on graph
        start_course = np.random.choice(courses_list)
        start_node = kg.course_nodes[start_course]
        
        # Get neighbors
        neighbors = list(kg.graph.successors(start_node))[:3]
        if neighbors:
            target_node = np.random.choice(neighbors)
            edge_data = kg.graph.get_edge_data(start_node, target_node)
            
            if edge_data:
                edge_type = edge_data.get('edge_type', '')
                
                if edge_type == 'prerequisite':
                    target_course = kg.graph.nodes[target_node].get('code', '')
                    query = f"What is a prerequisite for {start_course}?"
                    answer = f"{target_course} is a prerequisite for {start_course}."
                    
                    entities = extract_entities_from_query(query)
                    subgraph = retriever.retrieve_subgraph(query, entities, max_hops=2)
                    graph_context = retriever.format_subgraph_context(subgraph)
                    
                    synthetic.append({
                        'query': query,
                        'answer': answer,
                        'graph_context': graph_context,
                        'reasoning_path': f"{target_course} -> {start_course}",
                        'type': 'synthetic_prerequisite'
                    })
    
    return synthetic

# Generate synthetic questions
print("Generating synthetic questions...")
synthetic_questions = generate_synthetic_questions_from_graph(kg, num_synthetic=50)
print(f"✅ Generated {len(synthetic_questions)} synthetic questions")

# Augment existing questions
augmented = []
for q in multi_hop_questions[:20]:
    variations = augment_question(q['query'], q['answer'])
    for var in variations:
        var['graph_context'] = q.get('graph_context', '')
        var['reasoning_path'] = q.get('reasoning_path', '')
        augmented.append(var)

print(f"✅ Generated {len(augmented)} augmented question variations")


In [ ]:
# Save LoRA adapters
model.save_pretrained("lora_model_kg_qa")
tokenizer.save_pretrained("lora_model_kg_qa")
print("✅ LoRA adapters saved to 'lora_model_kg_qa'")

# Save knowledge graph
import pickle
with open("kg_graph.pkl", "wb") as f:
    pickle.dump(kg, f)
print("✅ Knowledge graph saved to 'kg_graph.pkl'")

# Save retriever (without GAT model for now)
with open("graph_retriever.pkl", "wb") as f:
    pickle.dump(retriever, f)
print("✅ Graph retriever saved to 'graph_retriever.pkl'")

# Optional: Export merged model
# model.save_pretrained_merged("merged_model_kg_qa", tokenizer, save_method="merged_16bit")
# print("✅ Merged model saved to 'merged_model_kg_qa'")


## 17. Summary and Next Steps

### What We've Built:

1. **Knowledge Graph**: Constructed from course data with nodes (courses, professors, topics) and edges (prerequisite, taught_by, covers_topic)
2. **Prerequisites & Topics Extraction**: Automated extraction from course descriptions
3. **Graph Attention Network**: Framework for graph-based retrieval (GAT model can be trained separately)
4. **Multi-hop Reasoning Data**: Generated complex questions requiring graph traversal
5. **RAG Training Format**: Combined graph context with LLM training
6. **Evaluation Framework**: Metrics for QA evaluation (EM, F1, BLEU, ROUGE)
7. **Inference Pipeline**: End-to-end query answering with graph retrieval
8. **Data Augmentation**: Synthetic question generation
9. **Performance Monitoring**: Tracking graph retrieval statistics

### Next Steps:

1. **Train GAT Model**: Implement and train the Graph Attention Network for better graph retrieval scoring
2. **Expand Test Set**: Create a comprehensive labeled test set with ground truth answers
3. **Fine-tune Hyperparameters**: Optimize GAT and LLM training parameters
4. **Add More Relationship Types**: Include degree requirements, course sequences, etc.
5. **Improve Topic Extraction**: Use better NLP models for topic extraction
6. **Multi-task Learning**: Jointly train graph retrieval and answer generation
7. **Attention Visualization**: Show which graph nodes the model attends to
8. **Production Deployment**: Create API endpoints for the inference pipeline
